In [1]:
import sys
from pathlib import Path

project_root = Path("..").resolve()
sys.path.append(str(project_root))


Import Feature Engineering Function

In [2]:
import pandas as pd

from src.feature_engineering import (
    load_data,
    create_aggregate_features,
    create_time_features,
    prepare_base_table,
    build_pipeline,
    run_feature_engineering
)


Load Raw Data

In [3]:
df = load_data()

df.head()


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15 02:18:49+00:00,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15 02:19:08+00:00,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15 02:44:21+00:00,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15 03:32:55+00:00,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15 03:34:21+00:00,2,0


In [4]:
df.shape

(95662, 16)

Create Aggregate Feature

In [5]:
agg_features = create_aggregate_features(df)
agg_features.head()

,CustomerId,total_amount,avg_amount,transaction_count,std_amount
0,CustomerId_1,-10000.0,-10000.000000,1,0.000000
1,CustomerId_10,-10000.0,-10000.000000,1,0.000000
2,CustomerId_1001,20000.0,4000.000000,5,6558.963333
3,CustomerId_1002,4225.0,384.090909,11,560.498966
4,CustomerId_1003,20000.0,3333.333333,6,6030.478146


In [6]:
agg_features.describe()

,total_amount,avg_amount,transaction_count,std_amount
count,3.742000e+03,3.742000e+03,3742.000000,3.742000e+03
mean,1.717377e+05,1.571562e+04,25.564404,1.360517e+04
std,2.717305e+06,1.676991e+05,96.929602,9.689344e+04
min,-1.049000e+08,-4.250000e+05,1.000000,0.000000e+00
25%,4.077438e+03,1.000000e+03,2.000000,5.011411e+02
50%,2.000000e+04,2.583846e+03,7.000000,3.184898e+03
75%,7.996775e+04,4.877614e+03,20.000000,6.745369e+03
max,8.345124e+07,8.601821e+06,4091.000000,3.309916e+06


Extracting Feature

In [8]:
df_time = create_time_features(df.copy())
df_time.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,transaction_hour,transaction_day,transaction_month,transaction_year
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15 02:18:49+00:00,2,0,2,15,11,2018
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15 02:19:08+00:00,2,0,2,15,11,2018
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15 02:44:21+00:00,2,0,2,15,11,2018
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15 03:32:55+00:00,2,0,3,15,11,2018
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15 03:34:21+00:00,2,0,3,15,11,2018


In [9]:
base_table,cat_cols,num_cols = prepare_base_table(df_time)
base_table.head()

,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductCategory,ChannelId,PricingStrategy,Value,transaction_hour,transaction_day,transaction_month,transaction_year
0,CustomerId_1,UGX,256,ProviderId_4,airtime,ChannelId_2,4,10000.000000,16.000000,21.000000,11.000000,2018
1,CustomerId_10,UGX,256,ProviderId_4,airtime,ChannelId_2,4,10000.000000,16.000000,21.000000,11.000000,2018
2,CustomerId_1001,UGX,256,ProviderId_5,financial_services,ChannelId_3,2,6080.000000,7.800000,16.000000,11.000000,2018
3,CustomerId_1002,UGX,256,ProviderId_5,airtime,ChannelId_3,4,434.090909,13.454545,14.727273,8.636364,2018
4,CustomerId_1003,UGX,256,ProviderId_5,financial_services,ChannelId_3,2,5333.333333,14.333333,1.000000,2.000000,2019


In [10]:
cat_cols,num_cols

(['CurrencyCode',
  'CountryCode',
  'ProviderId',
  'ProductCategory',
  'ChannelId',
  'PricingStrategy'],
 ['Value',
  'transaction_hour',
  'transaction_day',
  'transaction_month',
  'transaction_year'])

In [11]:
df_merged = base_table.merge(agg_features,on="CustomerId",how="left")
df_merged.head()

,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductCategory,ChannelId,PricingStrategy,Value,transaction_hour,transaction_day,transaction_month,transaction_year,total_amount,avg_amount,transaction_count,std_amount
0,CustomerId_1,UGX,256,ProviderId_4,airtime,ChannelId_2,4,10000.000000,16.000000,21.000000,11.000000,2018,-10000.0,-10000.000000,1,0.000000
1,CustomerId_10,UGX,256,ProviderId_4,airtime,ChannelId_2,4,10000.000000,16.000000,21.000000,11.000000,2018,-10000.0,-10000.000000,1,0.000000
2,CustomerId_1001,UGX,256,ProviderId_5,financial_services,ChannelId_3,2,6080.000000,7.800000,16.000000,11.000000,2018,20000.0,4000.000000,5,6558.963333
3,CustomerId_1002,UGX,256,ProviderId_5,airtime,ChannelId_3,4,434.090909,13.454545,14.727273,8.636364,2018,4225.0,384.090909,11,560.498966
4,CustomerId_1003,UGX,256,ProviderId_5,financial_services,ChannelId_3,2,5333.333333,14.333333,1.000000,2.000000,2019,20000.0,3333.333333,6,6030.478146


In [12]:
df_merged.shape

(3742, 16)

Run Feature Engineering Function

In [13]:
run_feature_engineering()

Feature engineering completed and saved.


In [14]:
processed_df = pd.read_csv("../data/processed/processed_data.csv")
processed_df.head()

,Value,transaction_hour,transaction_day,transaction_month,transaction_year,total_amount,avg_amount,transaction_count,std_amount,CurrencyCode_UGX,...,ProductCategory_utility_bill,ChannelId_ChannelId_1,ChannelId_ChannelId_2,ChannelId_ChannelId_3,ChannelId_ChannelId_5,PricingStrategy_0,PricingStrategy_1,PricingStrategy_2,PricingStrategy_4,CustomerId
0,-0.052297,0.883284,0.770545,1.038381,-0.829377,-0.066891,-0.153364,-0.253459,-0.140432,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,CustomerId_1
1,-0.052297,0.883284,0.770545,1.038381,-0.829377,-0.066891,-0.153364,-0.253459,-0.140432,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,CustomerId_10
2,-0.075710,-1.222654,0.050136,1.038381,-0.829377,-0.055849,-0.069870,-0.212186,-0.072731,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,CustomerId_1001
3,-0.109431,0.229556,-0.133241,0.523448,-0.829377,-0.061655,-0.091435,-0.150278,-0.134647,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,CustomerId_1002
4,-0.080169,0.455248,-2.111093,-0.922326,1.205724,-0.055849,-0.073846,-0.201868,-0.078186,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,CustomerId_1003


In [15]:
processed_df.shape

(3742, 35)

In [17]:
build_pipeline(cat_cols,num_cols)

,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The given string will be formatted using two field names: ``transformer_name`` and ``feature_name``. e.g. `